In [1]:
# Imports 
import numpy as np
import random
from datetime import datetime

In [2]:
# Parameters
n_cities = 20

n_population = 100

mutation_rate = 0.3

In [3]:
# Generating a list of coordenades representing each city
coordinates_list = [[x,y] for x,y in zip(np.random.randint(0,100,n_cities),np.random.randint(0,100,n_cities))]
names_list = np.array(['Berlin', 'London', 'Moscow', 'Barcelona', 'Rome', 'Paris', 'Vienna', 'Munich', 'Istanbul', 'Kyiv', 'Bucharest', 'Minsk', 'Warsaw', 'Budapest', 'Milan', 'Prague', 'Sofia', 'Birmingham', 'Brussels', 'Amsterdam'])
cities_dict = { x:y for x,y in zip(names_list,coordinates_list)}

# Function to compute the distance between two points
def compute_city_distance_coordinates(a,b):
    return ((a[0]-b[0])**2+(a[1]-b[1])**2)**0.5

def compute_city_distance_names(city_a, city_b, cities_dict):
    return compute_city_distance_coordinates(cities_dict[city_a], cities_dict[city_b])

cities_dict

{'Berlin': [96, 54],
 'London': [59, 9],
 'Moscow': [68, 0],
 'Barcelona': [72, 51],
 'Rome': [51, 86],
 'Paris': [40, 88],
 'Vienna': [52, 14],
 'Munich': [81, 88],
 'Istanbul': [9, 22],
 'Kyiv': [81, 62],
 'Bucharest': [57, 17],
 'Minsk': [98, 14],
 'Warsaw': [74, 42],
 'Budapest': [44, 39],
 'Milan': [69, 42],
 'Prague': [11, 1],
 'Sofia': [43, 66],
 'Birmingham': [75, 15],
 'Brussels': [51, 96],
 'Amsterdam': [75, 3]}

In [4]:
# First step: Create the first population set
def genesis(city_list, n_population):

    population_set = []
    for i in range(n_population):
        #Randomly generating a new solution
        sol_i = city_list[np.random.choice(list(range(n_cities)), n_cities, replace=False)]
        population_set.append(sol_i)
    return np.array(population_set)

population_set = genesis(names_list, n_population)
population_set

array([['Amsterdam', 'Rome', 'London', ..., 'Brussels', 'Sofia',
        'Munich'],
       ['Minsk', 'Prague', 'Rome', ..., 'Warsaw', 'Barcelona', 'Paris'],
       ['Kyiv', 'Brussels', 'Prague', ..., 'Amsterdam', 'Sofia',
        'Bucharest'],
       ...,
       ['Paris', 'Brussels', 'Sofia', ..., 'Amsterdam', 'Rome',
        'Istanbul'],
       ['Sofia', 'Birmingham', 'Bucharest', ..., 'Vienna', 'Istanbul',
        'Minsk'],
       ['Barcelona', 'Warsaw', 'Paris', ..., 'Vienna', 'Birmingham',
        'Minsk']], dtype='<U10')

In [5]:
def fitness_eval(city_list, cities_dict):
    total = 0
    for i in range(n_cities-1):
        a = city_list[i]
        b = city_list[i+1]
        total += compute_city_distance_names(a,b, cities_dict)
    return total

In [6]:
def get_all_fitnes(population_set, cities_dict):
    fitnes_list = np.zeros(n_population)

    #Looping over all solutions computing the fitness for each solution
    for i in  range(n_population):
        fitnes_list[i] = fitness_eval(population_set[i], cities_dict)

    return fitnes_list

fitnes_list = get_all_fitnes(population_set,cities_dict)
fitnes_list

array([ 930.04246968,  951.94357236, 1051.69451041,  837.13637572,
        948.73796133,  870.34217843,  959.49805808,  967.46843404,
       1032.81038931, 1052.17581401, 1065.65111037,  910.73189872,
        959.49257125,  970.19002228,  848.72519028,  886.46503335,
        910.28276442, 1055.91122784,  889.73424352,  930.892438  ,
       1098.13527306,  851.35615253, 1059.74122206,  955.04294411,
       1077.29217983,  902.40630636, 1019.18730124,  793.77797358,
       1096.48924732,  942.89953958,  977.05208072, 1144.61272721,
        970.98909402, 1038.72958785,  846.8811107 , 1051.08825877,
       1096.25191363, 1012.63445899,  926.43515544, 1071.40871397,
        990.52818132, 1057.23696345,  961.45288716,  951.60297513,
        869.38370458,  876.0188823 ,  913.29267136,  980.57712109,
        889.99733658,  932.17159209,  932.43199212, 1003.14224558,
        988.6448967 ,  907.61081119,  720.7356406 ,  919.24286063,
        985.71213777,  957.97189278,  909.82095832,  780.72327

In [7]:
def progenitor_selection(population_set,fitnes_list):
    total_fit = fitnes_list.sum()
    prob_list = fitnes_list/total_fit
    
    #Notice there is the chance that a progenitor. mates with oneself
    progenitor_list_a = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    progenitor_list_b = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    
    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]
    
    
    return np.array([progenitor_list_a,progenitor_list_b])


progenitor_list = progenitor_selection(population_set,fitnes_list)
progenitor_list[0][2]

array(['Berlin', 'Paris', 'Amsterdam', 'Bucharest', 'Budapest', 'Kyiv',
       'Istanbul', 'Rome', 'Milan', 'Warsaw', 'Munich', 'Sofia', 'Vienna',
       'Barcelona', 'Moscow', 'London', 'Prague', 'Birmingham',
       'Brussels', 'Minsk'], dtype='<U10')

In [8]:
def mate_progenitors(prog_a, prog_b):
    offspring = prog_a[0:5]

    for city in prog_b:

        if not city in offspring:
            offspring = np.concatenate((offspring,[city]))

    return offspring
            
    
    
def mate_population(progenitor_list):
    new_population_set = []
    for i in range(progenitor_list.shape[1]):
        prog_a, prog_b = progenitor_list[0][i], progenitor_list[1][i]
        offspring = mate_progenitors(prog_a, prog_b)
        new_population_set.append(offspring)
        
    return new_population_set

new_population_set = mate_population(progenitor_list)
new_population_set[0]

array(['Warsaw', 'Paris', 'Minsk', 'Istanbul', 'Birmingham', 'Brussels',
       'Sofia', 'Moscow', 'Vienna', 'Kyiv', 'Prague', 'Berlin',
       'Budapest', 'Rome', 'Barcelona', 'Bucharest', 'Munich',
       'Amsterdam', 'London', 'Milan'], dtype='<U10')

In [9]:
def mutate_offspring(offspring):
    for q in range(int(n_cities*mutation_rate)):
        a = np.random.randint(0,n_cities)
        b = np.random.randint(0,n_cities)

        offspring[a], offspring[b] = offspring[b], offspring[a]

    return offspring
    
    
def mutate_population(new_population_set):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring))
    return mutated_pop

mutated_pop = mutate_population(new_population_set)
mutated_pop[0]

array(['Bucharest', 'Prague', 'Minsk', 'Istanbul', 'Milan', 'Brussels',
       'Sofia', 'Moscow', 'Birmingham', 'Kyiv', 'Warsaw', 'Berlin',
       'Budapest', 'Rome', 'Barcelona', 'Paris', 'Munich', 'Amsterdam',
       'Vienna', 'London'], dtype='<U10')

In [10]:
best_solution = [-1,np.inf,np.array([])]
for i in range(10000):
    if i%50==0: print(i, best_solution[1], fitnes_list.mean(), datetime.now().strftime("%d/%m/%y %H:%M"))
    fitnes_list = get_all_fitnes(mutated_pop,cities_dict)
    
    #Saving the best solution
    if fitnes_list.min() < best_solution[1]:
        best_solution[0] = i
        best_solution[1] = fitnes_list.min()
        best_solution[2] = np.array(mutated_pop)[fitnes_list.min() == fitnes_list]
    
    progenitor_list = progenitor_selection(population_set,fitnes_list)
    new_population_set = mate_population(progenitor_list)
    
    mutated_pop = mutate_population(new_population_set)

0 inf 958.6463433105267 21/09/22 13:53
50 641.8946933508279 970.632602835083 21/09/22 13:53
100 640.2615120335417 968.9722543285862 21/09/22 13:53
150 584.066218453678 971.313012962522 21/09/22 13:53
200 584.066218453678 949.2070523106798 21/09/22 13:53
250 584.066218453678 971.1214217316494 21/09/22 13:53
300 584.066218453678 976.7235429827476 21/09/22 13:53
350 584.066218453678 970.183311020847 21/09/22 13:53
400 584.066218453678 973.4158825202218 21/09/22 13:53
450 584.066218453678 971.2420024257625 21/09/22 13:53
500 584.066218453678 964.818829013848 21/09/22 13:53
550 584.066218453678 969.8863784841662 21/09/22 13:53
600 584.066218453678 972.4950195040103 21/09/22 13:53
650 584.066218453678 977.0208127573378 21/09/22 13:53
700 584.066218453678 961.7603298912575 21/09/22 13:53
750 584.066218453678 967.7998117683509 21/09/22 13:53
800 584.066218453678 967.8449079476283 21/09/22 13:53
850 584.066218453678 981.0478758457107 21/09/22 13:53
900 584.066218453678 966.0952214458823 21/09/2

7500 556.498460218209 958.2582964190557 21/09/22 13:57
7550 556.498460218209 975.4619541784924 21/09/22 13:57
7600 556.498460218209 955.742405431347 21/09/22 13:57
7650 556.498460218209 968.754462480555 21/09/22 13:57
7700 556.498460218209 961.2264746339366 21/09/22 13:57
7750 556.498460218209 945.5882587042548 21/09/22 13:57
7800 556.498460218209 964.6216755252358 21/09/22 13:57
7850 556.498460218209 971.4672904456321 21/09/22 13:57
7900 556.498460218209 967.1944922903275 21/09/22 13:57
7950 556.498460218209 960.6914776944311 21/09/22 13:57
8000 556.498460218209 973.9471709208136 21/09/22 13:57
8050 556.498460218209 984.4952534760247 21/09/22 13:57
8100 556.498460218209 978.1227412641091 21/09/22 13:57
8150 556.498460218209 969.5810306865413 21/09/22 13:57
8200 556.498460218209 971.1338226862493 21/09/22 13:57
8250 556.498460218209 961.8148913448188 21/09/22 13:57
8300 556.498460218209 971.339912096419 21/09/22 13:58
8350 556.498460218209 969.4770159296444 21/09/22 13:58
8400 556.4984

In [11]:
best_solution

[1490,
 556.498460218209,
 array([['Rome', 'Paris', 'Sofia', 'Kyiv', 'Warsaw', 'Berlin', 'Minsk',
         'Amsterdam', 'Bucharest', 'Vienna', 'Moscow', 'London', 'Munich',
         'Brussels', 'Barcelona', 'Milan', 'Birmingham', 'Budapest',
         'Prague', 'Istanbul']], dtype='<U10')]